In [25]:
# (1) Single Allocation Implementation

## __new__ is called before __init__ 
import random

class Database:
    initialized = False

    def __init__(self):
        self.id = random.randint(1,101)
        # print('Generated an id of ', self.id)
        # print('Loading database from file')
        pass

    _instance = None

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super(Database, cls)\
                .__new__(cls, *args, **kwargs)

        return cls._instance


database = Database()

if __name__ == '__main__':
    d1 = Database()
    d2 = Database()

    print(d1.id, d2.id)
    print(d1 == d2)
    print(database == d1)

83 83
True
True


Python is Object oriented language, every thing is an object in python. Python is having special type of  methods called magic methods named with preceded and trailing double underscores.

When we talk about magic method __new__ we also need to talk about __init__

These methods will be called when you instantiate(The process of creating instance from class is called instantiation). That is when you create instance. The magic method __new__ will be called when instance is being created. Using this method you can customize the instance creation. This is only the method which will be called first then __init__ will be called to initialize instance when you are creating instance.

Method __new__ will take class reference as the first argument followed by arguments which are passed to constructor(Arguments passed to call of class to create instance). Method __new__ is responsible to create instance, so you can use this method to customize object creation. Typically method __new__ will return the created instance object reference. Method __init__ will be called once __new__ method completed execution.

You can create new instance of the class by invoking the superclass’s __new__ method using super. Something like super(currentclass, cls).__new__(cls, [,….])
https://howto.lintel.in/python-__new__-magic-method-explained/

In [26]:
# (2) Decorator Implementation

def singleton(class_):
    instances = {}

    def get_instance(*args, **kwargs):
        if class_ not in instances:
            instances[class_] = class_(*args, **kwargs)
        return instances[class_]

    return get_instance


@singleton
class Database:
    def __init__(self):
        print('Loading database')


if __name__ == '__main__':
    d1 = Database()
    d2 = Database()
    print(d1 == d2)
    

Loading database
True


In [27]:
# (3) Instantiate the DB from a singleton metaclass Recommended method 

class Singleton(type):
    """ Metaclass that creates a Singleton base type when called. """
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls)\
                .__call__(*args, **kwargs)
        return cls._instances[cls]


class Database(metaclass=Singleton):
    def __init__(self):
        print('Loading database')


if __name__ == '__main__':
    d1 = Database()
    d2 = Database()
    print(d1 == d2)

    

Loading database
True


In [28]:
## Monostate/Borg..many instances with shared attributes 
## inheriting from a monostate baseclass is also the recommended way 
# all members are static :)

class CEO:
    __shared_state = {
        'name': 'Steve',
        'age': 55
    }

    def __init__(self):
        self.__dict__ = self.__shared_state

    def __str__(self):
        return f'{self.name} is {self.age} years old'


class Monostate:
    _shared_state = {}

    def __new__(cls, *args, **kwargs):
        obj = super(Monostate, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj


class CFO(Monostate):
    def __init__(self):
        self.name = ''
        self.money_managed = 0

    def __str__(self):
        return f'{self.name} manages ${self.money_managed}bn'

if __name__ == '__main__':
    ceo1 = CEO()
    print(ceo1)

    ceo1.age = 66

    ceo2 = CEO()
    ceo2.age = 77
    print(ceo1)
    print(ceo2)

    ceo2.name = 'Tim'

    ceo3 = CEO()
    print(ceo1, ceo2, ceo3)

    cfo1 = CFO()
    cfo1.name = 'Sheryl'
    cfo1.money_managed = 1

    print(cfo1)

    cfo2 = CFO()
    cfo2.name = 'Ruth'
    cfo2.money_managed = 10
    print(cfo1, cfo2, sep='\n')
    
    

Steve is 55 years old
Steve is 77 years old
Steve is 77 years old
Tim is 77 years old Tim is 77 years old Tim is 77 years old
Sheryl manages $1bn
Ruth manages $10bn
Ruth manages $10bn


In [30]:
## how do we test singleton objects especially if they are realtime database objects 
## that change over time. 

import unittest


class Singleton(type):
    _instances = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super(Singleton, cls).__call__(*args, **kwargs)
        return cls._instances[cls]


class Database(metaclass=Singleton):
    def __init__(self):
        self.population = {}
        f = open('capitals.txt', 'r')
        lines = f.readlines()
        for i in range(0, len(lines), 2):
            self.population[lines[i].strip()] = int(lines[i + 1].strip())
        f.close()

## this Database() is live and changing in realtime so unittesting may be challenging 

class SingletonRecordFinder:
    def total_population(self, cities):
        result = 0
        for c in cities:
            result += Database().population[c]
        return result


class ConfigurableRecordFinder:
    def __init__(self, db):
        self.db = db

    def total_population(self, cities):
        result = 0
        for c in cities:
            result += self.db.population[c]
        return result

## can create this dummydatabase for purpose of testing.
class DummyDatabase:
    population = {
        'alpha': 1,
        'beta': 2,
        'gamma': 3
    }

    def get_population(self, name):
        return self.population[name]

# class SingletonTests(unittest.TestCase):
#     def test_is_singleton(self):
#         db = Database()
#         db2 = Database()
#         self.assertEqual(db, db2)

#     def test_singleton_total_population(self):
#         """ This tests on a live database :( """
#         rf = SingletonRecordFinder()
#         names = ['Seoul', 'Mexico City']
#         tp = rf.total_population(names)
#         self.assertEqual(tp, 17500000 + 17400000)  # what if these change?

#     ddb = DummyDatabase()

#     def test_dependent_total_population(self):
#         crf = ConfigurableRecordFinder(self.ddb)
#         self.assertEqual(
#             crf.total_population(['alpha', 'beta']),
#             3
#         )

# if __name__ == '__main__':
#     unittest.main()
    

In [33]:
ddb = DummyDatabase()
crf = ConfigurableRecordFinder(ddb)


In [34]:
crf.total_population(['alpha', 'beta'])


3

In [36]:
# Python factory function is used to create function objects. So a function works as a factory 
# to create and return function object. It can return different functions based
# on parameter of function. The returned function object can be later invoked and will
# have access to passed parameter because of closure.

from unittest import TestCase
from copy import deepcopy

def is_singleton(factory):
    x = factory()
    y = factory()
    return x is y

class Evaluate(TestCase):
    def test_exercise(self):
        obj = [1, 2, 3]
        self.assertTrue(is_singleton(lambda: obj))
        self.assertFalse(is_singleton(lambda: deepcopy(obj)))